In [3]:
import os
import pickle

import numpy as np
import pandas as pd

from scipy.optimize import curve_fit
from importlib import reload

from tasks import flux, fluxseries

In [ ]:
reload(flux)

# Flux

In [ ]:
path_ds = os.path.join('/Users','thiago','Neurodata','Datasets','flux_dsB1/')
cal = ('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec')
listSubj = np.array(os.listdir(path_ds))
listSubj = listSubj[np.logical_not([n.startswith('.') for n in listSubj])]
listSubj = listSubj[[os.path.isdir(os.path.join(path_ds,n)) for n in listSubj]]
listSubj.sort()
listSubj

## dfList (table with session summary)

In [ ]:
dfVI.head()

In [ ]:
dicList = {'subj':[],'date':[],'IRI':[],'VI':[],'cued':[],'deplete':[],'dur':[],\
          'meanA':[],'meanB':[],'meanC':[],'rewSize':[],'fname':[],'sessname':[]}

cal = ('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec')

listInt=(30,60,120)

for iSubj in range(len(listSubj)):
    nameSubj = listSubj[iSubj]
    print(iSubj,nameSubj)

    pathSess = os.path.join(path_ds,nameSubj)
    listSess = os.listdir(pathSess)
    
    listSess.sort()
    listSess=np.array(listSess)
    listSess=listSess[[ n.endswith('.pkl') for n in listSess ]]

    for iSess in range(len(listSess)):
        
        fname=os.path.join(pathSess,listSess[iSess])
        with open(fname,'rb') as filehandler:
            sess = pickle.load(filehandler)
            
        sessname = sess.fname.split('_')
        date=sessname[3] + str(np.where([n == sessname[2][0:3] for n in cal])[0][0]+1).zfill(2) + sessname[2][3:6]
        
        dicList['IRI'].append(int(sess.params['IRI']))
        
        ndxVI=np.logical_and(dfVI.subj==nameSubj,dfVI.date==date)
        dicList['meanA'].append(listInt[np.array(list(set(dfVI[np.logical_and(ndxVI,dfVI.armNo==0)].armVal))).item()])
        dicList['meanB'].append(listInt[np.array(list(set(dfVI[np.logical_and(ndxVI,dfVI.armNo==1)].armVal))).item()])
        dicList['meanC'].append(listInt[np.array(list(set(dfVI[np.logical_and(ndxVI,dfVI.armNo==2)].armVal))).item()])
        
        dicList['VI'].append(bool(sess.params['VI']))
        
        dicList['rewSize'].append(int(sess.params['rewFirst']))

        dicList['sessname'].append(sess.fname)
        dicList['subj'].append(sessname[0])
        dicList['date'].append(date)
        dicList['fname'].append(fname)

        dicList['dur'].append(int((sess.dfPokes['tsPoke'].iloc[-1]-sess.dfPokes['tsPoke'].iloc[0])/60))
        if 'Cued' in sess.params.keys():
            dicList['cued'].append(bool(sess.params['Cued']))
        else:
            dicList['cued'].append(False)
            
        if 'Deplete' in sess.params.keys():
            dicList['deplete'].append(bool(sess.params['Deplete']))
        else:
            dicList['deplete'].append(True)
            continue

In [ ]:
dfSessList=pd.DataFrame(dicList)

In [ ]:
with open(os.path.join('/Users','thiago','Neurocode','Scripts_TaskSuite','3AF_report','dfSessList.pkl'),'wb') as fhandle:
    pickle.dump(dfSessList,fhandle)

## dfVI (table of intervals)

In [ ]:
dicVI = {'armNo':[],'interval':[],'subj':[], 'date':[],'ndx':[]}

ndxOffset=0
for iSubj in range(len(listSubj)):
    nameSubj=listSubj[iSubj]
    if not os.path.isdir(os.path.join(path_ds,nameSubj)):
        continue
    listSess = np.array(os.listdir(os.path.join(path_ds,nameSubj)))
    listSess = listSess[[n.endswith('.pkl') for n in listSess]]
    listSess.sort()
    for iSess in range(len(listSess)):
        nameSess = listSess[iSess]

        with open(os.path.join(path_ds,nameSubj,nameSess), 'rb') as filehandler:
            sess = pickle.load(filehandler)

        sessname = sess.fname.split('_')
        
        for iArm in set(sess.dfRwd['arm']):
            dfRwd = sess.dfRwd.loc[sess.dfRwd['arm']==iArm]
            dfSetup = sess.dfSetup.loc[sess.dfSetup['arm']==iArm]
            ndx=dfRwd.index
            for iRew in range(len(ndx)-1):
                tsRwd=dfRwd['tsRwd'].loc[ndx[iRew]]                
                dicVI['armNo'].append(iArm)
                dicVI['interval'].append(dfSetup['tsSetup'].loc[dfSetup['tsSetup']>tsRwd].min()-tsRwd)
                dicVI['subj'].append(nameSubj)
                dicVI['date'].append(sessname[3] + str(np.where([n == sessname[2][0:3] for n in cal])[0][0]+1).zfill(2) + sessname[2][3:6])
                dicVI['ndx'].append(ndx[iRew]+ndxOffset)
        ndxOffset=max(dicVI['ndx'])+1
        

dfVI = pd.DataFrame(dicVI)
dfVI=dfVI.set_index('ndx')
dfVI=dfVI.sort_index()
dfVI=dfVI[['subj', 'date','armNo','interval']]

In [ ]:
dfVI['armVal']=np.nan

for iSubj in range(len(listSubj)):
    nameSubj=listSubj[iSubj]
    ndxSubj=dfVI['subj']==nameSubj
    listSess = np.array(list(set(dfVI[ndxSubj].date)))
    listSess.sort()
    for iSess in range(len(listSess)):
        nameSess = listSess[iSess]
        ndxSess=np.logical_and(ndxSubj,dfVI['date']==nameSess)
        mu=np.full(3,np.nan)
        sigma=np.full(3,np.nan)
        armVal=np.full(3,np.nan)
        for iArm in set(dfVI[ndxSess]['armNo']):
            ndxArm=np.logical_and(ndxSess,dfVI['armNo']==iArm)
            mu[iArm]=dfVI[ndxArm]['interval'].mean()
            sigma[iArm]=dfVI[ndxArm]['interval'].std()
        for iArm in set(dfVI[ndxSess]['armNo']):
            try:
                assert(np.arange(3)[mu[iArm]==sorted(mu)].item()==np.arange(3)[sigma[iArm]==sorted(sigma)].item())
            except:
                print(nameSubj,nameSess)
            
            ndxArm=np.logical_and(ndxSess,dfVI['armNo']==iArm)
            dfVI.iloc[np.arange(len(dfVI))[ndxArm],np.arange(len(dfVI.columns))[dfVI.columns=='armVal'].item()]=np.arange(3)[mu[iArm]==sorted(mu)].item()

dfVI.armVal=dfVI.armVal.astype(int)

In [ ]:
with open(os.path.join('/Users','thiago','Neurocode','Scripts_TaskSuite','3AF_report','dfVI.pkl'),'wb') as fhandle:
    pickle.dump(dfVI,fhandle)

## dfLing (table of time spent per visit)

In [ ]:
dicLing = {'armNo':[],'armVal':[],'lingert':[],'nRew':[],'subj':[], 'date':[]}

for iSubj in range(len(listSubj)):
    nameSubj=listSubj[iSubj]
    if not os.path.isdir(os.path.join(path_ds,nameSubj)):
        continue
    listSess = np.array(os.listdir(os.path.join(path_ds,nameSubj)))
    listSess = listSess[[n.endswith('.pkl') for n in listSess]]
    listSess.sort()
    for iSess in range(len(listSess)):
        nameSess = listSess[iSess]

        with open(os.path.join(path_ds,nameSubj,nameSess), 'rb') as filehandler:
            sess = pickle.load(filehandler)

        sessname = sess.fname.split('_')
        tsPoke = sess.dfPokes['tsPoke']
        arm = sess.dfPokes['arm']
        
        MeanABC=sess.params.index[[n.startswith('Mean') for n in sess.params.index]]
        ndxSwi=np.where(sess.dfPokes['isSwitch'])[0]
        
        for i in range(len(ndxSwi)-1):
            assert(arm.iloc[ndxSwi[i+1]-1]==arm.iloc[ndxSwi[i]])
            dicLing['armNo'].append(arm.iloc[ndxSwi[i]])
            dicLing['armVal'].append(sess.params[MeanABC][arm.iloc[ndxSwi[i]]])
            dicLing['lingert'].append(tsPoke.iloc[ndxSwi[i+1]-1]-tsPoke.iloc[ndxSwi[i]])
            dicLing['subj'].append(nameSubj)
            dicLing['date'].append(sessname[3] + str(np.where([n == sessname[2][0:3] for n in cal])[0][0]+1).zfill(2) + sessname[2][3:6])
            dicLing['nRew'].append(sess.dfPokes['isRwded'].iloc[ndxSwi[i]:ndxSwi[i+1]].sum())

dfLing = pd.DataFrame(dicLing)
dfLing=dfLing[['subj', 'date','armNo','armVal','nRew','lingert']]
dfLing=dfLing.sort_values(['subj','date'])

In [ ]:
with open(os.path.join('/Users','thiago','Neurocode','Scripts_TaskSuite','3AF_report','dfLing.pkl'),'wb') as fhandle:
    pickle.dump(dfLing,fhandle)

## dfTince (table of elapsed time since rwds at time of disengagement)

dicTince = {'arm':[],'nextArm':[],'nextRwded':[],'tinceR0':[],'tinceR1':[],'tinceR2':[],'travel':[],'subj':[],'date':[]}
off=[]
for iSubj in range(len(listSubj)):
    nameSubj=listSubj[iSubj]
    if not os.path.isdir(os.path.join(path_ds,nameSubj)):
        continue
    listSess = np.array(os.listdir(os.path.join(path_ds,nameSubj)))
    listSess = listSess[[n.endswith('b.pkl') for n in listSess]]
    listSess.sort()
    for iSess in range(len(listSess)):
        nameSess = listSess[iSess]

        with open(os.path.join(path_ds,nameSubj,nameSess), 'rb') as filehandler:
            sess = pickle.load(filehandler)

        sessname = sess.fname.split('_')
        
        dfPokes=sess.dfPokes.sort_values('tsPoke')
        dfRwd=sess.dfRwd.sort_values('tsRwd')
        
        for i in dfRwd.index:
            try:
                assert(dfPokes.loc[i][dfPokes.loc[i].isRwded].arm.item()==dfRwd.loc[i].arm.item())
                

In [ ]:
dicTince = {'arm':[],'nextArm':[],'nextRwded':[],'tinceR0':[],'tinceR1':[],'tinceR2':[],'travel':[],'subj':[],'date':[]}
off=[]
for iSubj in range(len(listSubj)):
    nameSubj=listSubj[iSubj]
    if not os.path.isdir(os.path.join(path_ds,nameSubj)):
        continue
    listSess = np.array(os.listdir(os.path.join(path_ds,nameSubj)))
    listSess = listSess[[n.endswith('.pkl') for n in listSess]]
    listSess.sort()
    for iSess in range(len(listSess)):
        nameSess = listSess[iSess]
        with open(os.path.join(path_ds,nameSubj,nameSess), 'rb') as filehandler:
            sess = pickle.load(filehandler)

        sessname = sess.fname.split('_')
        
        dfPokes=sess.dfPokes.sort_values('tsPoke')
        dfRwd=sess.dfRwd.sort_values('tsRwd')

        tsAllgood=max([dfRwd['tsRwd'][dfRwd.arm==0].min(),dfRwd['tsRwd'][dfRwd.arm==1].min(),dfRwd['tsRwd'][dfRwd.arm==2].min()])
#         break
#     break
# for iPoke in [100]:
        for iPoke in range(len(dfPokes)-1):
            currArm=dfPokes['arm'].iloc[iPoke]
            nextArm=dfPokes['arm'].iloc[iPoke+1]
            tsCurrPoke=dfPokes['tsPoke'].iloc[iPoke]
            tsNextPoke=dfPokes['tsPoke'].iloc[iPoke+1]

            conds=np.array([])
            conds=np.append(conds,nextArm!=currArm) #last response of a sequence
        #     conds=np.append(conds,np.logical_and(dfRwd.arm==currArm,dfRwd['tsRwd']<tsCurrPoke).any()) #any prior reward at this arm
            conds=np.append(conds,tsCurrPoke>tsAllgood) #
            if conds.all():
                date=sessname[3] + str(np.where([n == sessname[2][0:3] for n in cal])[0][0]+1).zfill(2) + sessname[2][3:6]
                dicTince['date'].append(date)
                
                dicTince['tinceR0'].append(tsCurrPoke-dfRwd['tsRwd'][np.logical_and(dfRwd.arm==0,dfRwd.tsRwd<tsCurrPoke)].max())
                dicTince['tinceR1'].append(tsCurrPoke-dfRwd['tsRwd'][np.logical_and(dfRwd.arm==1,dfRwd.tsRwd<tsCurrPoke)].max())
                dicTince['tinceR2'].append(tsCurrPoke-dfRwd['tsRwd'][np.logical_and(dfRwd.arm==2,dfRwd.tsRwd<tsCurrPoke)].max())
                
                ndxVI=np.logical_and(dfVI.subj==nameSubj,dfVI.date==date)
                currArmVal=np.array(list(set(dfVI[np.logical_and(ndxVI,dfVI.armNo==currArm)].armVal))).item()
                nextArmVal=np.array(list(set(dfVI[np.logical_and(ndxVI,dfVI.armNo==nextArm)].armVal))).item()
                dicTince['arm'].append(currArmVal)
                dicTince['nextArm'].append(nextArmVal)
                dicTince['travel'].append(tsNextPoke-tsCurrPoke)
                dicTince['subj'].append(nameSubj)
                dicTince['nextRwded'].append(np.any([dfRwd.tsRwd==tsNextPoke]))
                if not np.any([dfRwd.tsRwd==tsNextPoke]):
                    off.append(tsNextPoke-dfRwd.tsRwd[abs(dfRwd.tsRwd-tsNextPoke)==min(abs(dfRwd.tsRwd-tsNextPoke))].item())
#         break
#     break

dfTince = pd.DataFrame(dicTince)
dfTince=dfTince[['subj', 'date','arm','nextArm','nextRwded','tinceR0','tinceR1','tinceR2','travel']]
dfTince=dfTince.sort_values(['subj','date'])

In [ ]:
dfTince

In [ ]:
with open(os.path.join('/Users','thiago','Neurocode','Scripts_TaskSuite','3AF_report','dfTince.pkl'),'wb') as fhandle:
    pickle.dump(dfTince,fhandle)

## dfSanPr (table with estimated and observed $p(reward)$)

In [ ]:
dicSanPr = {'arm':[],'tince':[],'isRwded':[],'subj':[],'date':[],'shouldRwded':[]}#,'lambda_inv':[]
nViol=0
nViolShu=0
                
for iSubj in range(len(listSubj)):
    nameSubj=listSubj[iSubj]
    if not os.path.isdir(os.path.join(path_ds,nameSubj)):
        continue
    listSess = np.array(os.listdir(os.path.join(path_ds,nameSubj)))
    listSess = listSess[[n.endswith('.pkl') for n in listSess]]
    listSess.sort()
    for iSess in range(len(listSess)):
        nameSess = listSess[iSess]

        with open(os.path.join(path_ds,nameSubj,nameSess), 'rb') as filehandler:
            sess = pickle.load(filehandler)

        sessname = sess.fname.split('_')
        
        dfPokes=sess.dfPokes.sort_values('tsPoke')
        dfRwd=sess.dfRwd.sort_values('tsRwd')
        dfSetup=sess.dfSetup.sort_values('tsSetup')

        tsAllgood=max([dfRwd['tsRwd'][dfRwd.arm==0].min(),dfRwd['tsRwd'][dfRwd.arm==1].min(),dfRwd['tsRwd'][dfRwd.arm==2].min()])
        
        for iPoke in np.arange(len(dfPokes))[np.logical_and(dfPokes.isSwitch,dfPokes.tsPoke>tsAllgood)]:

            currArm=dfPokes.arm.iloc[iPoke]
            currTsPoke=dfPokes.tsPoke.iloc[iPoke]
            tsLastRwdHere=dfRwd[np.logical_and(dfRwd.arm==currArm,dfRwd.tsRwd<currTsPoke)].tsRwd.max()
            
            assert(np.logical_and(dfPokes.isSwitch.iloc[iPoke],currTsPoke>tsAllgood))
            
            ndxVI=np.logical_and(dfVI.subj==nameSubj,dfVI.date==date)
            currArmVal=np.array(list(set(dfVI[np.logical_and(ndxVI,dfVI.armNo==currArm)].armVal))).item()
            dicSanPr['arm'].append(currArmVal)

            ndx=np.logical_and(dfRwd.arm==currArm,dfRwd.tsRwd<currTsPoke)
            dicSanPr['tince'].append(currTsPoke-dfRwd[ndx].tsRwd.max())

            dicSanPr['shouldRwded'].append(dfSetup[np.logical_and(dfSetup.arm==currArm,dfSetup.tsSetup>tsLastRwdHere)].tsSetup.min()<currTsPoke)
            if dicSanPr['shouldRwded'][-1]:
                try:
                    assert(dfSetup[np.isclose(currTsPoke,dfSetup.tsSetup)].arm.item()==currArm)
                except:
                    nViolShu=nViolShu+1

#             dicSanPr['isRwded'].append(np.isclose(currTsPoke,dfRwd.tsRwd).any())
            dicSanPr['isRwded'].append(dfPokes.iloc[iPoke].isRwded)
            if dicSanPr['isRwded'][-1]:
                try:
                    assert(dfRwd[np.isclose(currTsPoke,dfRwd.tsRwd)].arm.item()==currArm)
                except:
                    nViol=nViol+1

            dicSanPr['subj'].append(nameSubj)
            
            date=sessname[3] + str(np.where([n == sessname[2][0:3] for n in cal])[0][0]+1).zfill(2) + sessname[2][3:6]
            dicSanPr['date'].append(date)
#             break            
#         break
#     break

In [ ]:
print(nViol,nViolShu)
dfSanPr = pd.DataFrame(dicSanPr)
dfSanPr=dfSanPr[['subj', 'date','arm','tince','isRwded','shouldRwded']]
print('Hamming distance: ' + str((dfSanPr.isRwded.values!=dfSanPr.shouldRwded.values).mean()))

# dfSanPr=dfSanPr.sort_values(['subj','date'])

In [ ]:
with open(os.path.join('/Users','thiago','Neurocode','Scripts_TaskSuite','3AF_report','dfSanPr.pkl'),'wb') as fhandle:
    pickle.dump(dfSanPr,fhandle)

## FluxSeries

In [5]:
reload(fluxseries)

<module 'tasks.fluxseries' from '/Users/thiago/Programs/TaskSuite/tasks/fluxseries.py'>

In [36]:
m0=10.
m1=1.
n=np.array((3,6,12))
print("Multiplicative factors for depletion rate: ",(m1/m0)**(1./n))

Multiplicative factors for depletion rate:  [0.46415888 0.68129207 0.82540419]


In [7]:
path_ds = os.path.join('/Users','thiago','Neurodata','Datasets','fluxseries_ds3/')
cal = ('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec')
listSubj = np.array(os.listdir(path_ds))
# listSubj = listSubj[[n.endswith('.pkl') for n in listSubj]]
listSubj = listSubj[[not n.startswith('.') for n in listSubj]]
listSubj.sort()
listSubj

array(['TG023', 'TG024', 'TG027', 'TG028'], dtype='<U9')

In [29]:
dicDepl = {'arm':[],'nRews':[],'subj':[], 'date':[]}

for iSubj in range(len(listSubj)):
    nameSubj=listSubj[iSubj]
    if not os.path.isdir(os.path.join(path_ds,nameSubj)):
        continue
    listSess = np.array(os.listdir(os.path.join(path_ds,nameSubj)))
    listSess = listSess[[n.endswith('.pkl') for n in listSess]]
    listSess.sort()
    for iSess in range(len(listSess)):
        nameSess = listSess[iSess]

        with open(os.path.join(path_ds,nameSubj,nameSess), 'rb') as filehandler:
            sess = pickle.load(filehandler)       

        sessname = sess.fname.split('_')
        date=sessname[3] + str(np.where([n == sessname[2][0:3] for n in cal])[0][0]+1).zfill(2) + sessname[2][3:6]

        listFirsts = np.arange(len(sess.dfRwd))[np.append(np.array(True),np.diff(sess.dfRwd.arm)!=0)]

        for iFirst in range(len(listFirsts)-1):

            dicDepl['arm'].append(sess.dfRwd.iloc[listFirsts[iFirst]].arm)
            dicDepl['nRews'].append(listFirsts[iFirst+1]-listFirsts[iFirst])
            dicDepl['subj'].append(nameSubj)
            dicDepl['date'].append(date)
        

In [30]:
dfDeplt=pd.DataFrame(dicDepl)

In [31]:
with open(os.path.join('/Users','thiago','Neurocode','Scripts_TaskSuite','3AF_report','dfDeplt.pkl'),'wb') as fhandle:
    pickle.dump(dfDeplt,fhandle)

In [33]:
sess.params

Cued           0
Deplete        1
IRI            6
Ports_ABC    123
Series       ABC
rewA          30
rewB          30
rewC          30
rewFirst      20
rewLast        2
rewN_A         3
rewN_B         6
rewN_C        12
dtype: object